In [ ]:
import requests
import pandas as pd
import time
import random
from tqdm import tqdm
from bs4 import BeautifulSoup
import re

### Lấy category id

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 Edg/128.0.0.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi,en;q=0.9,en-US;q=0.8',
    'Referer': 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789',
    'x-guest-token': 'f1kXJZDbVQ8N0PFzH3E92yKUeg4Wa6wY',
    'Connection': 'keep-alive',
}

params = {
    'limit': '40',
    'include': 'advertisement',
    'aggregations': '2',
    'trackity_id': '563cc097-0278-3210-d9eb-2b08ef7b99a0',
    'category': '1789',
    'page': '1',
}

In [ ]:
categories_id = []
repo = requests.get("https://tiki.vn/", headers=headers, params=params)
soup = BeautifulSoup(repo.text, 'html.parser')

soup = soup.find('div', class_='styles__StyledListItem-sc-w7gnxl-0 cjqkgR')
a = soup.find_all('a')
for i in a:
    match = re.search(r'/c(\d+)', i['href'])
    categories_id.append(match.group(1))

categories_id

### Lấy id của sản phẩm

In [ ]:
product_id = []
for j in categories_id:
    params['category'] = j
    for i in range(1, 2):
        params['page'] = i
        response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers=headers, params=params)
        if response.status_code == 200:
            # print("requests complete")
            for record in response.json().get('data'):
                product_id.append({'id': record.get('id')})
print(len(product_id))

In [ ]:
df = pd.DataFrame(product_id)
df = set(df['id'])
df = pd.DataFrame(df)
df.rename(columns={0: 'id'}, inplace=True)
print(df)

### Lấy comment trên từng sản phẩm

In [ ]:
import requests
import pandas as pd
import time
import random
from tqdm import tqdm

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi,en;q=0.9,en-US;q=0.8',
    'Referer': 'https://tiki.vn/ke-toan-via-he-thuc-hanh-bao-cao-tai-chinh-can-ban-tu-quay-ban-nuoc-chanh-p262977989.html?',
    'x-guest-token': 'pUYHub5GIMswKcLtmv6gzl9yDFo8Cd7E',
    'Connection': 'keep-alive',
    'TE': 'Trailers',
}

params = {
    'product_id': '',
    'sort': 'score|desc,id|desc,stars|all',
    'page': '1',
    'limit': '10',
    'include': 'comments'
}

def comment_parser(json):
    d = dict()
    d['id'] = json.get(id)
    d['title'] = json.get('title')
    d['content'] = json.get('content')
    d['thank_count'] = json.get('thank_count')
    d['customer_id']  = json.get('customer_id')
    d['rating'] = json.get('rating')
    d['created_at'] = json.get('created_at')
    d['customer_name'] = json.get('created_by').get('full_name')
    d['purchased_at'] = json.get('created_by').get('purchased_at')
    return d

p_ids = df.id.to_list()[:10]
result = []
for pid in tqdm(p_ids, total=len(p_ids)):
    params['product_id'] = pid
    print('Crawl comment for product {}'.format(pid))
    for i in range(1):
        params['page'] = i
        yes = 1
        while(yes):
            try:
                response = requests.get('https://tiki.vn/api/v2/reviews?', headers=headers, params=params)
                if response.status_code == 200:
                    for comment in response.json().get('data'):
                        result.append(comment_parser(comment))
                    print('Crawl comment page {} success!!!'.format(i))
                yes = 0
            except:
                print('Crawl data {} fail !!!'.format(pid))
        
df_comment = pd.DataFrame(result)
df_comment.to_csv('comments_data_ncds.csv', index=False)

In [ ]:
response = requests.get('https://tiki.vn/api/v2/reviews?', headers=headers, params=params)
print(response.status_code)

In [ ]:
df_comment

### Lấy thông tin sản phẩm

In [ ]:
import pandas as pd
import requests
import time
import random
from tqdm import tqdm

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi-VN,vi;q=0.8,en-US;q=0.5,en;q=0.3',
    'Referer': 'https://tiki.vn/dien-thoai-samsung-galaxy-m31-128gb-6gb-hang-chinh-hang-p58259141.html?src=category-page-1789&2hi=0',
    'x-guest-token': '8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY',
    'Connection': 'keep-alive',
    'TE': 'Trailers',
}

params = (
    ('platform', 'web'),
    ('spid', 187266106)
)

def parser_product(json):
    d = dict()
    d['id'] = json.get('id')
    d['sku'] = json.get('sku')
    d['short_description'] = json.get('short_description')
    d['price'] = json.get('price')
    d['list_price'] = json.get('list_price')
    d['discount'] = json.get('discount')
    d['discount_rate'] = json.get('discount_rate')
    d['review_count'] = json.get('review_count')
    d['all_time_quantity_sold'] = json.get('all_time_quantity_sold')
    d['inventory_status'] = json.get('inventory_status')
    d['stock_item_qty'] = json.get('stock_item').get('qty')
    d['stock_item_max_sale_qty'] = json.get('stock_item').get('max_sale_qty')
    d['product_name'] = json.get('name')
    d['brand_id'] = json.get('brand').get('id')
    d['brand_name'] = json.get('brand').get('name')
    return d

p_ids = df.id.to_list()
print(p_ids)
result = []
for pid in tqdm(p_ids, total=len(p_ids)):
    print(pid)
    yes = 1
    while(yes):
        try:
            response = requests.get('https://tiki.vn/api/v2/products/{}'.format(pid), headers=headers, params=params)
            print('Crawl data {} success !!!'.format(pid))
            result.append(parser_product(response.json()))
            yes = 0
        except:
            print('Crawl data {} fail !!!'.format(pid))
        
df_product = pd.DataFrame(result)
df_product.to_csv('crawled_data_ncds.csv', index=False)

In [ ]:
print(result)

In [21]:
from pyspark.sql import SparkSession

# Tạo SparkSession với cấu hình kết nối đến HDFS trên container thông qua localhost
spark = SparkSession.builder \
    .appName("Write to HDFS") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .getOrCreate()

In [22]:
# Ví dụ tạo DataFrame
data = [("Alice", 25), ("Bob", 30)]
columns = ["name", "age"]
df = spark.createDataFrame(data, columns)

# Đường dẫn trên HDFS (đảm bảo đường dẫn và cổng đã được map trong docker-compose)
output_path = "hdfs://localhost:9000/user/airflow/comments/mydata.parquet"

# Ghi DataFrame vào HDFS ở định dạng Parquet
df.write.mode("overwrite").parquet(output_path)

print("File đã được ghi vào HDFS tại:", output_path)

spark.stop()

Py4JJavaError: An error occurred while calling o240.parquet.
: org.apache.hadoop.ipc.RpcException: RPC response has invalid length
	at org.apache.hadoop.ipc.Client$IpcStreams.readResponse(Client.java:1933)
	at org.apache.hadoop.ipc.Client$Connection.receiveRpcResponse(Client.java:1238)
	at org.apache.hadoop.ipc.Client$Connection.run(Client.java:1134)


In [23]:
spark.stop()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("List HDFS Folders") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://172.18.0.6:9000") \
    .getOrCreate()
spark

In [ ]:
# Đọc file Parquet
try:
    df = spark.read.parquet("hdfs://172.18.0.6:9000/user/airflow/comments/comment_inmemory_20250311_145408.parquet")
    df.show()
except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")

# Đóng SparkSession
spark.stop()

In [ ]:
from pyspark.sql import SparkSession

# Tạo SparkSession với cấu hình HDFS
spark = SparkSession.builder \
    .appName("List HDFS Folders") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .getOrCreate()


# Lấy đối tượng FileSystem từ Hadoop configuration
fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())

# Đường dẫn gốc trên HDFS
path = spark._jvm.org.apache.hadoop.fs.Path("hdfs://localhost:9000/user/airflow/comments")

# Liệt kê các file và thư mục trong đường dẫn
file_status = fs.listStatus(path)


print("Các folder trên HDFS:")
for status in file_status:
    print(status.getPath().toString())

# Dừng SparkSession khi xong
spark.stop()